[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EMQA_spark_spread/EMQA_spark_spread.ipynb)

# EMQA_spark_spread
Spark spread: Gas vs Electricity -- profitability of gas-fired power generation.
**Output:** `spark_spread.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
import yfinance as yf

def fetch(ticker, start='2020-01-01', end='2025-12-31'):
    d = yf.download(ticker, start=start, end=end, progress=False)
    if isinstance(d.columns, pd.MultiIndex):
        return d['Close'].squeeze().dropna()
    return d['Close'].dropna()


In [ ]:
import os
def load_elec():
    paths = [
        '../../charts/electricity_cache.csv',
        '/Users/danielpele/Documents/Energy MBA/charts/electricity_cache.csv',
    ]
    for p in paths:
        if os.path.exists(p):
            df = pd.read_csv(p, parse_dates=[0], index_col=0)
            return df
    raise FileNotFoundError("No electricity data found")

elec = load_elec()
if 'price' in elec.columns:
    price_col = 'price'
elif 'Price' in elec.columns:
    price_col = 'Price'
else:
    price_col = elec.columns[0]
elec_price = elec[price_col].dropna()


In [ ]:
# Fetch Natural Gas
gas = fetch('NG=F')

# Align electricity and gas on common dates
elec_daily = elec_price.copy()
elec_daily.index = pd.to_datetime(elec_daily.index)
gas.index = pd.to_datetime(gas.index)
common = elec_daily.index.intersection(gas.index)

if len(common) < 30:
    # If not enough overlap, use approximate alignment
    elec_daily = elec_daily.resample('B').last().dropna()
    gas_aligned = gas.reindex(elec_daily.index, method='ffill').dropna()
    common = elec_daily.index.intersection(gas_aligned.index)
    elec_aligned = elec_daily.loc[common]
    gas_aligned = gas_aligned.loc[common]
else:
    elec_aligned = elec_daily.loc[common]
    gas_aligned = gas.loc[common]

# Spark spread = Electricity price - Gas / heat_rate
# Heat rate ~ 7 MMBtu/MWh for a typical gas plant
heat_rate = 7.0
# Gas is in $/MMBtu, Electricity in EUR/MWh (approximate conversion)
spark = elec_aligned.values - gas_aligned.values * heat_rate

spark_series = pd.Series(spark, index=common)
print(f"Spark spread: mean={spark_series.mean():.2f}, std={spark_series.std():.2f}")

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Panel 1 - Both series
ax1 = axes[0]
ln1 = ax1.plot(elec_aligned.index, elec_aligned.values, color=COLORS['orange'], linewidth=1, label='Electricity')
ax1.set_ylabel('Electricity Price', color=COLORS['orange'])
ax2 = ax1.twinx()
ln2 = ax2.plot(gas_aligned.index, gas_aligned.values, color=COLORS['blue'], linewidth=1, label='Natural Gas')
ax2.set_ylabel('Gas Price ($/MMBtu)', color=COLORS['blue'])
ax2.spines['top'].set_visible(False)
lns = ln1 + ln2
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc='upper center', bbox_to_anchor=(0.5, -0.05), frameon=False, ncol=2)
ax1.set_title('Electricity and Natural Gas Prices')

# Panel 2 - Spark spread with bands
sp_mean = spark_series.mean()
sp_std = spark_series.std()
axes[1].plot(spark_series.index, spark_series.values, color=COLORS['green'], linewidth=1, label='Spark Spread')
axes[1].axhline(sp_mean, color=COLORS['blue'], linewidth=1.5, label=f'Mean ({sp_mean:.1f})')
axes[1].axhline(sp_mean + sp_std, color=COLORS['red'], linestyle='--', linewidth=1, alpha=0.7)
axes[1].axhline(sp_mean - sp_std, color=COLORS['red'], linestyle='--', linewidth=1, alpha=0.7)
axes[1].fill_between(spark_series.index, sp_mean - sp_std, sp_mean + sp_std,
                      alpha=0.1, color=COLORS['red'])
axes[1].axhline(0, color=COLORS['gray'], linestyle=':', linewidth=0.8)
axes[1].set_ylabel('Spark Spread')
axes[1].set_xlabel('Date')
axes[1].set_title(f'Spark Spread (Elec - Gas x {heat_rate} heat rate)')
axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), frameon=False, ncol=3)

plt.tight_layout()
save_fig(fig, 'spark_spread.pdf')
plt.show()